# Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/


### Input Data
- One timeseries txt file per participant. Conditions must be separated in unique folder
- Need a txt file of the labels for each ROI

### Outputs
- Circle graph showing thresholded connectivity of each ROI. ROI color denotes modules. 
- Circle graph of module edge weights. 
- CSV file with module assignment & nodal metrics for each ROI

### Running the notebook
The following parts of the code should be changed
1. Filepath to timeseries 
2. Name of circle graph figure
3. Module dict to assign to module graph
4. Name of module graph figure
5. Name of csv file

## Note: 
### Print Statements are commented out to save time, remove comments if desired. 

In [1]:
# Imports 
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import statistics
import pdb
from scipy import stats
import scipy
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj, ColorbarObj
from visbrain.io import download_file


/anaconda3/lib/python3.7/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [2]:
basepath = '/Users/jennygilbert/Documents/'

## Load in the data

### Find the path to the data

In [3]:
#Find the path to data
file_list = glob.glob(os.path.join(basepath,'betaseries_bevel/4_combine_timeseries/choice/*.txt'))

In [4]:
#Check the files found
print(file_list)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-017_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-020_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-075_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-042_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-069_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-078_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-087_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-053_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-064_choice.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/choice/sub-031_choice.txt', '/Users/jennygilbert/Document

In [5]:
#Check to see how many participants 
#len(file_list)

### Make a dictionary to read in the files to pandas

In [6]:
#Setting the ditionary
my_dict={}
for item in file_list:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-017_choice
sub-020_choice
sub-075_choice
sub-042_choice
sub-069_choice
sub-078_choice
sub-087_choice
sub-053_choice
sub-064_choice
sub-031_choice
sub-006_choice
sub-082_choice
sub-028_choice
sub-061_choice
sub-056_choice
sub-003_choice
sub-034_choice
sub-025_choice
sub-012_choice
sub-047_choice
sub-070_choice
sub-039_choice
sub-009_choice
sub-088_choice
sub-022_choice
sub-015_choice
sub-040_choice
sub-077_choice
sub-066_choice
sub-004_choice
sub-033_choice
sub-018_choice
sub-085_choice
sub-054_choice
sub-063_choice
sub-036_choice
sub-001_choice
sub-080_choice
sub-048_choice
sub-059_choice
sub-010_choice
sub-027_choice
sub-072_choice
sub-045_choice
sub-029_choice
sub-083_choice
sub-060_choice
sub-057_choice
sub-002_choice
sub-035_choice
sub-024_choice
sub-013_choice
sub-046_choice
sub-071_choice
sub-038_choice
sub-016_choice
sub-021_choice
sub-074_choice
sub-043_choice
sub-068_choice
sub-086_choice
sub-079_choice
sub-052_choice
sub-030_choice
sub-007_choice
sub-055_choice
sub-062_ch

In [7]:
# Checking to make sure its populated
#my_dict['sub-024_reward']

### Read the data from the dictionary into pandas

In [8]:
#Setting the data dictionary
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

In [9]:
#Check the dictionary
data_dict.keys()

dict_keys(['sub-017_choice', 'sub-020_choice', 'sub-075_choice', 'sub-042_choice', 'sub-069_choice', 'sub-078_choice', 'sub-087_choice', 'sub-053_choice', 'sub-064_choice', 'sub-031_choice', 'sub-006_choice', 'sub-082_choice', 'sub-028_choice', 'sub-061_choice', 'sub-056_choice', 'sub-003_choice', 'sub-034_choice', 'sub-025_choice', 'sub-012_choice', 'sub-047_choice', 'sub-070_choice', 'sub-039_choice', 'sub-009_choice', 'sub-088_choice', 'sub-022_choice', 'sub-015_choice', 'sub-040_choice', 'sub-077_choice', 'sub-066_choice', 'sub-004_choice', 'sub-033_choice', 'sub-018_choice', 'sub-085_choice', 'sub-054_choice', 'sub-063_choice', 'sub-036_choice', 'sub-001_choice', 'sub-080_choice', 'sub-048_choice', 'sub-059_choice', 'sub-010_choice', 'sub-027_choice', 'sub-072_choice', 'sub-045_choice', 'sub-029_choice', 'sub-083_choice', 'sub-060_choice', 'sub-057_choice', 'sub-002_choice', 'sub-035_choice', 'sub-024_choice', 'sub-013_choice', 'sub-046_choice', 'sub-071_choice', 'sub-038_choice',

In [10]:
#Check for the dataframe
#data_dict['sub-058_reward']

### Create a new dictionary with correlation matrix

In [11]:
#Setting up the correlation dictionary
cor_dict={}

for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix
    

In [12]:
#check the dictionary
#list(cor_dict.values())[3]

### Make a dictionary of labels for the nodes

In [13]:
#This points to a txt file with the ROI names in a list separated by commas
path = os.path.join(basepath,'betaseries_bevel/5_analysis/labels.txt')
df_label = pd.read_csv(path, sep=',')

#df_label.head()

In [14]:
labels_dict = {}
n=0
for item in df_label:
    labels_dict[n]=item
    n=n+1

In [15]:
print(labels_dict)

{0: 'Amygdala_L', 1: 'Amygdala_R', 2: 'Dorsal_striatum_L', 3: 'Dorsal_striatum_R', 4: 'Fusiform_gyrus_L', 5: 'Fusiform_gyrus_R', 6: 'Hippocampus_L', 7: 'Hippocampus_R', 8: 'Insula_L', 9: 'Insula_R', 10: 'Intracalcarine_cortex_L', 11: 'Intracalcarine_cortex_R', 12: 'lOFC_L', 13: 'lOFC_R', 14: 'mOFC_L', 15: 'mOFC_R', 16: 'Oral_somatosensory_cortex_R', 17: 'Oral_somatosensory_cortex_L', 18: 'Precuneus_L', 19: 'Precuneus_R', 20: 'Ventral_striatum_L', 21: 'Ventral_striatum_R', 22: 'vlPFC_L', 23: 'vlPFC_R', 24: 'vlThalamus_L', 25: 'vlThalamus_R', 26: 'vmPFC_L', 27: 'vmPFC_R'}


### Function to create a graph with positive or negative values and minimum correlation value

In [16]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)
- The modules (communitites in the network at the participant level

## Changes from the First Iteration

1. Test for mean,sd, and module assignment distribution among participant's graphs

In [17]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    sd_dict={}
    graph_dict={}
    partition_dict={}
    for key, values in list_o_data.items():
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            sd=x.std()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            sd_dict.setdefault(key, []).append(sd)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'stdev':sd_dict, 'graphs':graph_dict,'modules':partition_dict})

### Apply the function to correlations & check output

In [18]:
# Apply function
GRAPHS = make_graphs(cor_dict, "positive", 0)

In [19]:
# Check the keys for the dictionary
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'stdev', 'graphs', 'modules'])

In [20]:
# Check modules for one subject
#GRAPHS['stdev']['sub-001_punish']

In [21]:
#Check to make sure graphs are filled
#Test = GRAPHS['graphs']['sub-001_reward'][0]
#Test.edges(data=True)

In [22]:
GRAPHS['mean_FC']

{'sub-017_choice': [0.2546083375608922],
 'sub-020_choice': [0.2290242918230263],
 'sub-075_choice': [0.3322615065468891],
 'sub-042_choice': [0.3072267791382684],
 'sub-069_choice': [0.30880305348483406],
 'sub-078_choice': [0.28257408425795233],
 'sub-087_choice': [0.34505876076227104],
 'sub-053_choice': [0.34364685255765903],
 'sub-064_choice': [0.39691787678627344],
 'sub-031_choice': [0.33459652346063273],
 'sub-006_choice': [0.2878053096641184],
 'sub-082_choice': [0.3018395432595864],
 'sub-028_choice': [0.31605014745279497],
 'sub-061_choice': [0.33920866495359603],
 'sub-056_choice': [0.26617619430273337],
 'sub-003_choice': [0.31861805583677777],
 'sub-034_choice': [0.3087461624519029],
 'sub-025_choice': [0.30953429141527006],
 'sub-012_choice': [0.32944718554227714],
 'sub-047_choice': [0.23474618319223545],
 'sub-070_choice': [0.31451038683432114],
 'sub-039_choice': [0.29499849552916374],
 'sub-009_choice': [0.2697558052104211],
 'sub-088_choice': [0.3506892137765341],
 

### Get the standard deviation of the mean correlation

In [23]:
#statistics.stdev(GRAPHS['mean_FC'])
m=[]

for key,value in GRAPHS['mean_FC'].items():
    m.append(value[0])

In [24]:
#describe the mean correlation of participant's graphs
stats.describe(m)

DescribeResult(nobs=85, minmax=(0.21181484375293963, 0.6302920206113318), mean=0.30915886762795464, variance=0.0049395280558069965, skewness=1.8636574236452013, kurtosis=4.805002579316959)

In [25]:
statistics.mean(m)

0.30915886762795464

In [26]:
#m

In [27]:
#Show the histogram of average correlation across participant's graphs

hist, bin_edges = scipy.histogram(m, bins = 10) 
  
# Checking the results 
print ("No. of points in each bin : ", hist) 
print ("Size of the bins          : ", bin_edges) 
  
# plotting the histogram 
plt.bar(bin_edges[:-1], hist, width = .1) 
plt.xlim(min(bin_edges), max(bin_edges)) 
#plt.show() 

No. of points in each bin :  [16 24 26 10  2  3  2  1  0  1]
Size of the bins          :  [0.21181484 0.25366256 0.29551028 0.337358   0.37920571 0.42105343
 0.46290115 0.50474887 0.54659659 0.5884443  0.63029202]


(0.21181484375293963, 0.6302920206113318)

### Get the mean SD of participant's corrleation matricies

In [28]:
sd=[]
for key,value in GRAPHS['stdev'].items():
    sd.append(value[0])

In [29]:
#describe the mean correlation of participant's graphs
stats.describe(sd)

DescribeResult(nobs=85, minmax=(0.18973353425986, 0.3097375191241389), mean=0.22664519962028268, variance=0.00048191047892613757, skewness=1.6173239426271104, kurtosis=2.938124600759523)

In [30]:
#sd

In [31]:
hist, bin_edges = scipy.histogram(sd, bins = 10) 
  
# Checking the results 
print ("No. of points in each bin : ", hist) 
print ("Size of the bins          : ", bin_edges) 
  
# plotting the histogram 
plt.bar(bin_edges[:-1], hist, width = .05) 
plt.xlim(min(bin_edges), max(bin_edges)) 
#plt.show() 

No. of points in each bin :  [ 5 16 31 19  3  5  1  2  2  1]
Size of the bins          :  [0.18973353 0.20173393 0.21373433 0.22573473 0.23773513 0.24973553
 0.26173593 0.27373632 0.28573672 0.29773712 0.30973752]


(0.18973353425986, 0.3097375191241389)

If this value is low (<0.1) then you don't need to threshold the graph? 

### Test Modularity

Goal: use modularity function from communities to identify module structure that emerge during reward within the sample

In [32]:
#this is pulling out the module structure for each participant
modules=[]

for key,value in GRAPHS['modules'].items():
    modules.append(value[0])

In [33]:
#print it to make sure it works
modules

[{0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 0,
  5: 2,
  6: 0,
  7: 3,
  8: 2,
  9: 2,
  10: 2,
  11: 2,
  12: 2,
  13: 2,
  14: 4,
  15: 1,
  16: 5,
  17: 5,
  18: 4,
  19: 3,
  20: 6,
  21: 7,
  22: 5,
  23: 5,
  24: 7,
  25: 7,
  26: 3,
  27: 3},
 {0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 0,
  5: 0,
  6: 0,
  7: 1,
  8: 0,
  9: 0,
  10: 2,
  11: 2,
  12: 2,
  13: 2,
  14: 3,
  15: 3,
  16: 3,
  17: 4,
  18: 3,
  19: 4,
  20: 1,
  21: 1,
  22: 3,
  23: 2,
  24: 2,
  25: 2,
  26: 3,
  27: 5},
 {0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 0,
  5: 1,
  6: 1,
  7: 1,
  8: 3,
  9: 3,
  10: 4,
  11: 4,
  12: 4,
  13: 4,
  14: 4,
  15: 4,
  16: 2,
  17: 2,
  18: 0,
  19: 0,
  20: 0,
  21: 0,
  22: 3,
  23: 3,
  24: 4,
  25: 2,
  26: 5,
  27: 5},
 {0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 2,
  5: 3,
  6: 0,
  7: 0,
  8: 4,
  9: 4,
  10: 3,
  11: 3,
  12: 3,
  13: 4,
  14: 5,
  15: 6,
  16: 1,
  17: 3,
  18: 6,
  19: 3,
  20: 6,
  21: 6,
  22: 5,
  23: 4,
  24: 4,
  25: 3,
  26: 5,
  27: 1},
 {0: 0,
  1: 1,
  2: 2,
  3:

In [34]:
#made a df of the modules
df = pd.DataFrame.from_dict(modules)

In [35]:
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,0,1,1,0,2,0,3,2,2,...,4,3,6,7,5,5,7,7,3,3
1,0,0,1,1,0,0,0,1,0,0,...,3,4,1,1,3,2,2,2,3,5
2,0,1,2,2,0,1,1,1,3,3,...,0,0,0,0,3,3,4,2,5,5
3,0,0,1,1,2,3,0,0,4,4,...,6,3,6,6,5,4,4,3,5,1
4,0,1,2,0,3,0,3,3,4,1,...,4,4,0,0,4,4,3,3,0,0
5,0,1,2,3,0,0,0,4,4,2,...,4,1,2,2,6,4,1,2,1,3
6,0,1,0,0,2,2,2,3,4,0,...,3,3,1,1,0,0,2,4,1,0
7,0,1,2,3,0,0,4,4,0,0,...,3,3,5,5,2,2,0,1,3,1
8,0,1,2,2,3,3,4,4,2,2,...,4,1,3,3,2,2,2,4,2,2
9,0,1,2,2,3,3,3,3,2,1,...,0,2,0,1,2,2,3,1,2,2


In [36]:
#label the modules
df.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_R", 17:"Oral_somatosensory_cortex_L", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0,0,1,1,0,2,0,3,2,2,...,4,3,6,7,5,5,7,7,3,3
1,0,0,1,1,0,0,0,1,0,0,...,3,4,1,1,3,2,2,2,3,5
2,0,1,2,2,0,1,1,1,3,3,...,0,0,0,0,3,3,4,2,5,5
3,0,0,1,1,2,3,0,0,4,4,...,6,3,6,6,5,4,4,3,5,1
4,0,1,2,0,3,0,3,3,4,1,...,4,4,0,0,4,4,3,3,0,0
5,0,1,2,3,0,0,0,4,4,2,...,4,1,2,2,6,4,1,2,1,3
6,0,1,0,0,2,2,2,3,4,0,...,3,3,1,1,0,0,2,4,1,0
7,0,1,2,3,0,0,4,4,0,0,...,3,3,5,5,2,2,0,1,3,1
8,0,1,2,2,3,3,4,4,2,2,...,4,1,3,3,2,2,2,4,2,2
9,0,1,2,2,3,3,3,3,2,1,...,0,2,0,1,2,2,3,1,2,2


# Plot the distibution of module assignments for the ROIs

In [37]:
#df.hist()

### Only a couple ROI have stable module assignment -- modules 0, 1, and 2 
- 0:"Amygdala_L"
- 1:"Amygdala_R"
- 2:"Dorsal_striatum_L"

In [38]:
#find the mean over the columns
df.mean(axis = 0)

0     0.000000
1     0.470588
2     1.317647
3     1.482353
4     1.705882
5     2.047059
6     1.294118
7     1.294118
8     2.117647
9     2.164706
10    2.611765
11    2.647059
12    2.835294
13    2.800000
14    2.576471
15    2.658824
16    1.952941
17    1.894118
18    2.611765
19    1.823529
20    2.858824
21    2.800000
22    2.717647
23    2.647059
24    2.352941
25    2.282353
26    2.482353
27    2.364706
dtype: float64

In [39]:
# Check the standard deviation over the columns
df.std(axis = 0)

0     0.000000
1     0.502096
2     0.581700
3     0.780899
4     1.242343
5     1.387923
6     1.462414
7     1.478606
8     1.614014
9     1.639157
10    1.440024
11    1.485693
12    1.646403
13    1.660465
14    1.583971
15    1.644275
16    1.610279
17    1.380334
18    2.030096
19    1.760523
20    2.006851
21    2.045902
22    1.763464
23    1.716210
24    1.894264
25    1.823209
26    1.637447
27    1.798459
dtype: float64



## The standard deviation of module assignments is high (>1) for the majority of the modules. 

### This suggests the module assignment is not stable across participants 

### Using the median module assignment across participants makes more sense than the mean

In [40]:
#find the mean over the columns
partition_median = df.median()
df_median = pd.DataFrame(partition_median)

In [41]:
# View the median module for each node
df_median

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,2.0
5,2.0
6,1.0
7,1.0
8,2.0
9,2.0


### Make a Dicitionary with the modules

In [42]:
#Convert modules to dictionary
#mod_dict={}
#for index, row in df_median.iterrows():
    #mod_dict[index]= row[0]

#mod_dict

### Combine participant correlation matrices into one mean OR MEDIAN correlation matrix

In [43]:
#make the function to combine
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.median(x, axis=(2))
    #mu=np.mean(x, axis=(2))
    return(mu)

In [44]:
#Make the mean graph with correlations
mean_graph = make_total_graphs(GRAPHS['correlations'])

In [45]:
#Check to make sure this worked 
print(mean_graph.shape)

#Convert the graph to a numpy matrix so it can be recognized by networkX
mean_graph_mat = np.matrix(mean_graph)

#Check the mean correlation to use to threshold later
mean_graph_mat.mean()

(28, 28)


0.285400937050057

In [46]:
mean_graph_mat.std()

0.16488388929642636

In [47]:
A = mean_graph_mat.flatten()
B = np.squeeze(np.asarray(A))
stats.skew(B)

3.057201015879242

In [48]:
meanG = nx.from_numpy_matrix(mean_graph_mat)

# Check Modularity in the Mean Graph

### This is the way Cohen's group usually calculates modularity, because its more stable

In [49]:
partition = community.best_partition(meanG)

In [50]:
#Add the modules and ROI labels to the graph
nx.set_node_attributes(meanG, partition, 'modules')
nx.set_node_attributes(meanG, labels_dict, 'ROIs')

In [51]:
# Check to make sure this worked
ROIs=nx.get_node_attributes(meanG,'ROIs')
ROIs

{0: 'Amygdala_L',
 1: 'Amygdala_R',
 2: 'Dorsal_striatum_L',
 3: 'Dorsal_striatum_R',
 4: 'Fusiform_gyrus_L',
 5: 'Fusiform_gyrus_R',
 6: 'Hippocampus_L',
 7: 'Hippocampus_R',
 8: 'Insula_L',
 9: 'Insula_R',
 10: 'Intracalcarine_cortex_L',
 11: 'Intracalcarine_cortex_R',
 12: 'lOFC_L',
 13: 'lOFC_R',
 14: 'mOFC_L',
 15: 'mOFC_R',
 16: 'Oral_somatosensory_cortex_R',
 17: 'Oral_somatosensory_cortex_L',
 18: 'Precuneus_L',
 19: 'Precuneus_R',
 20: 'Ventral_striatum_L',
 21: 'Ventral_striatum_R',
 22: 'vlPFC_L',
 23: 'vlPFC_R',
 24: 'vlThalamus_L',
 25: 'vlThalamus_R',
 26: 'vmPFC_L',
 27: 'vmPFC_R'}

In [52]:
#Check if this worked
partitions=nx.get_node_attributes(meanG,'modules')
partitions

{0: 0,
 1: 0,
 2: 1,
 3: 1,
 4: 0,
 5: 2,
 6: 0,
 7: 0,
 8: 3,
 9: 3,
 10: 2,
 11: 2,
 12: 4,
 13: 4,
 14: 2,
 15: 2,
 16: 1,
 17: 1,
 18: 5,
 19: 5,
 20: 6,
 21: 6,
 22: 1,
 23: 1,
 24: 3,
 25: 3,
 26: 1,
 27: 1}

# Module Connectivity

In [53]:
#Apply the partition to participant's data
#modg = community.induced_graph(partitions, meanG) 

### Function to make a thresholded graph

In [54]:
def threshold(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

In [55]:
threshG = threshold(meanG, 'positive', 0.3)

In [56]:
meanG.nodes(data=True)

NodeDataView({0: {'modules': 0, 'ROIs': 'Amygdala_L'}, 1: {'modules': 0, 'ROIs': 'Amygdala_R'}, 2: {'modules': 1, 'ROIs': 'Dorsal_striatum_L'}, 3: {'modules': 1, 'ROIs': 'Dorsal_striatum_R'}, 4: {'modules': 0, 'ROIs': 'Fusiform_gyrus_L'}, 5: {'modules': 2, 'ROIs': 'Fusiform_gyrus_R'}, 6: {'modules': 0, 'ROIs': 'Hippocampus_L'}, 7: {'modules': 0, 'ROIs': 'Hippocampus_R'}, 8: {'modules': 3, 'ROIs': 'Insula_L'}, 9: {'modules': 3, 'ROIs': 'Insula_R'}, 10: {'modules': 2, 'ROIs': 'Intracalcarine_cortex_L'}, 11: {'modules': 2, 'ROIs': 'Intracalcarine_cortex_R'}, 12: {'modules': 4, 'ROIs': 'lOFC_L'}, 13: {'modules': 4, 'ROIs': 'lOFC_R'}, 14: {'modules': 2, 'ROIs': 'mOFC_L'}, 15: {'modules': 2, 'ROIs': 'mOFC_R'}, 16: {'modules': 1, 'ROIs': 'Oral_somatosensory_cortex_R'}, 17: {'modules': 1, 'ROIs': 'Oral_somatosensory_cortex_L'}, 18: {'modules': 5, 'ROIs': 'Precuneus_L'}, 19: {'modules': 5, 'ROIs': 'Precuneus_R'}, 20: {'modules': 6, 'ROIs': 'Ventral_striatum_L'}, 21: {'modules': 6, 'ROIs': 'Vent

### Function to visualize thresholded graph with modules in colors

In [57]:
def jenny_graph(graph):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items()) #if your modules are named different change here
    nodes, positions = zip(*nx.get_node_attributes(graph,'ROIs').items())
    #positions
    positions=nx.circular_layout(graph) #this is defining a circluar graph, if you want a different one you change the circular part of this line
    
    #Figure size
    plt.figure(figsize=(40,25))
    
    
    #draws nodes
    color = np.array(color)
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=color,
                           node_size=1000,
                           alpha=0.8, 
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, labels = ROIs, font_size=10, 
                            font_family='sans-serif', fontweight = 'bold')
    
    
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.bwr #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    wt=weights*5
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Modularity',fontsize = 25)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)

    plt.axis('off')
    plt.title("Modularity and Edge Weights of Average Graph", fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/modularity_circle_reward.png", format="PNG")
    plt.show()

In [ ]:
jenny_graph(threshG)

# USE BCTPY FOR PARTICIPATION COEFFICIENT

In [59]:
import bct

In [60]:
#Participation coefficient usage
#bct.participation_coef(W, ci, degree='undirected')¶

In [61]:
b = np.array(list(partition.keys()))

In [62]:
cor_mat = np.array(mean_graph_mat)
test_array=np.array(list(list(b)))
testPART=np.vstack(test_array)
PC=bct.participation_coef(W=cor_mat, ci= testPART)

In [63]:
PC

array([0.95148723, 0.95112435, 0.9545654 , 0.95492351, 0.95207791,
       0.95034143, 0.95471504, 0.95240073, 0.95627904, 0.95479624,
       0.95007691, 0.95140366, 0.95224869, 0.95271317, 0.9517258 ,
       0.95187023, 0.95359225, 0.95530658, 0.95233218, 0.95261972,
       0.94635849, 0.94757001, 0.95280464, 0.95126172, 0.95382012,
       0.9541087 , 0.95434975, 0.95371131])

### Make Module Graph Connectivity Weights

Make a new graph with modules from whole group analysis 

In [64]:
modg = community.induced_graph(partitions, meanG)

In [65]:
nx.edges(modg, nbunch=None)

EdgeView([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 3), (3, 4), (3, 5), (3, 6), (4, 4), (4, 5), (4, 6), (5, 5), (5, 6), (6, 6)])

In [66]:
nx.get_edge_attributes(modg, "weight")

{(0, 0): 8.714637201176956,
 (0, 1): 8.53094309064918,
 (0, 2): 5.514512238527564,
 (0, 3): 5.224085190342246,
 (0, 4): 2.0133612936231415,
 (0, 5): 2.2342854794506963,
 (0, 6): 2.2788394589352534,
 (1, 1): 20.319023892666426,
 (1, 2): 8.823238287508719,
 (1, 3): 9.769050493600457,
 (1, 4): 3.695172621395049,
 (1, 5): 4.561126347815209,
 (1, 6): 2.9687573053894782,
 (2, 2): 8.888148171867407,
 (2, 3): 4.47342532117988,
 (2, 4): 2.81453867517229,
 (2, 5): 1.880743572719416,
 (2, 6): 1.6218451017957067,
 (3, 3): 6.524800877179274,
 (3, 4): 2.16515748023712,
 (3, 5): 1.6488853580998037,
 (3, 6): 1.5209100882859758,
 (4, 4): 2.4266604097670594,
 (4, 5): 0.7751550656759099,
 (4, 6): 0.6401791801704356,
 (5, 5): 2.377483340429849,
 (5, 6): 0.9960551923815344,
 (6, 6): 2.476146587580304}

In [67]:
# Stats

#nx.clustering(modg, weight=True)

### Function to make module structure graph

In [68]:
def module_fig(G):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())
    
    
    positions=nx.circular_layout(G)
    plt.figure(figsize=(25,20))
    
    color = np.array(list(G.nodes))
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    nx.draw_networkx_nodes(G,positions, 
                           node_color=color, 
                           node_size=1000,
                           alpha=1.0, 
                           cmap= 'Spectral',
                           vmin=cm,vmax=cM )
   
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, font_family='sans-serif')
    
    x=nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   
    edge_bar=plt.colorbar(x)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)
    
    plt.title("Module Connectivity Weights", fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/results/modularity_edges_reward_weighted.png"), format="PNG")
    plt.axis('off')
    plt.show()

In [69]:
module_fig(modg)

### Calculate Nodal Metrics by Participant
- Clustering Coefficient
- Betweenness centrality

Function to pull the nodal parameters by subject

In [ ]:
def make_node_stats(list_o_data, direction, min_cor):
    clustering_dict ={}
    centrality_dict ={}
    pc_dict ={}
    
    for key, values in list_o_data.items():
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            tG = create_corr_network_5(G, direction,min_cor)
            ########################################
            clustering = nx.clustering(tG, weight=True)
            ########################################
            clustering_dict.setdefault(key, []).append(clustering)
            ########################################
            centrality = nx.betweenness_centrality(tG, weight=True)
            ########################################
            centrality_dict.setdefault(key, []).append(centrality)
            ########################################
            cor_mat = np.array(cor_matrix)
            test_array=np.array(list(list(b)))
            testPART=np.vstack(test_array)
            PC=bct.participation_coef(W=cor_mat, ci= testPART)
            pc_dict.setdefault(key, []).append(PC)
            ########################################
    return({'clustering_coeff':clustering_dict, 'btn_centrality':centrality_dict, 'pc': pc_dict})

In [ ]:
#Metrics computed on an UNTHRESHOLDED GRAPH
nodes = make_node_stats(cor_dict, "positive", 0)

In [ ]:
nodes.keys()

In [ ]:
#nodes['pc']['sub-001_punish']

In [ ]:
pc=[]

for key,value in nodes['pc'].items():
    pc.append(value[0])

In [ ]:
df_pc = pd.DataFrame.from_dict(pc)

In [ ]:
#CHOICE
#df_pc['module0_mean'] = df_pc[[0,1,4,6,7]].mean(axis=1)
#df_pc['module1_mean'] = df_pc[[2,3,16,17]].mean(axis=1)
#df_pc['module2_mean'] = df_pc[[5,10,11,14,15]].mean(axis=1)
#df_pc['module3_mean'] = df_pc[[8,9,24,25]].mean(axis=1)
#df_pc['module4_mean'] = df_pc[[12,13]].mean(axis=1)
#df_pc['module5_mean'] = df_pc[[18,19]].mean(axis=1)
#df_pc['module6_mean'] = df_pc[[20,21]].mean(axis=1)
#df_pc['module7_mean'] = df_pc[[22,23,26,27]].mean(axis=1)

In [ ]:
#REWARD 
#df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
#df_cent['module1_mean'] = df_cent[[2,3,16,17]].mean(axis=1)
#df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
#df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
#df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
#df_cent['module5_mean'] = df_cent[[18,19,26,27]].mean(axis=1)
#df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)
#df_cent['module7_mean'] = df_cent[[22,23]].mean(axis=1)

In [ ]:
#PUNISHMENT
df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
df_cent['module1_mean'] = df_cent[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
df_cent['module5_mean'] = df_cent[[18,19]].mean(axis=1)
df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)

In [ ]:
df_pc.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

In [ ]:
clustering_coeff=[]

for key,value in nodes['clustering_coeff'].items():
    clustering_coeff.append(value[0])

In [ ]:
#bring into pandas
df_clust = pd.DataFrame.from_dict(clustering_coeff)

In [ ]:
#Calculate average module centrality *** CHOICE
#df_clust['module0_mean'] = df_clust[[0,1,4,6,7]].mean(axis=1)
#df_clust['module1_mean'] = df_clust[[2,3,16,17]].mean(axis=1)
#df_clust['module2_mean'] = df_clust[[5,10,11,14,15]].mean(axis=1)
#df_clust['module3_mean'] = df_clust[[8,9,24,25]].mean(axis=1)
#df_clust['module4_mean'] = df_clust[[12,13]].mean(axis=1)
#df_clust['module5_mean'] = df_clust[[18,19]].mean(axis=1)
#df_clust['module6_mean'] = df_clust[[20,21]].mean(axis=1)
#df_clust['module7_mean'] = df_clust[[22,23,26,27]].mean(axis=1)

In [ ]:
#Calculate average module centrality ***REWARD
#df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
#df_cent['module1_mean'] = df_cent[[2,3,16,17]].mean(axis=1)
#df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
#df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
#df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
#df_cent['module5_mean'] = df_cent[[18,19,26,27]].mean(axis=1)
#df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)
#df_cent['module7_mean'] = df_cent[[22,23]].mean(axis=1)

In [ ]:
#PUNISHMENT
df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
df_cent['module1_mean'] = df_cent[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
df_cent['module5_mean'] = df_cent[[18,19]].mean(axis=1)
df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)

In [ ]:

df_clust.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

In [ ]:
central=[]

for key,value in nodes['btn_centrality'].items():
    central.append(value[0])

In [ ]:
df_cent = pd.DataFrame.from_dict(central)

In [ ]:
#REWARD
#df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
#df_cent['module1_mean'] = df_cent[[2,3,16,17]].mean(axis=1)
#df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
#df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
#df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
#df_cent['module5_mean'] = df_cent[[18,19,26,27]].mean(axis=1)
#df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)
#df_cent['module7_mean'] = df_cent[[22,23]].mean(axis=1)

In [ ]:
#PUNISHMENT
df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
df_cent['module1_mean'] = df_cent[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
df_cent['module5_mean'] = df_cent[[18,19]].mean(axis=1)
df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)

In [ ]:
#CHOICE
#df_cent['module0_mean'] = df_cent[[0,1,4,6,7]].mean(axis=1)
#df_cent['module1_mean'] = df_cent[[2,3,16,17]].mean(axis=1)
#df_cent['module2_mean'] = df_cent[[5,10,11,14,15]].mean(axis=1)
#df_cent['module3_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
#df_cent['module4_mean'] = df_cent[[12,13]].mean(axis=1)
#df_cent['module5_mean'] = df_cent[[18,19]].mean(axis=1)
#df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)
#df_cent['module7_mean'] = df_cent[[22,23,26,27]].mean(axis=1)

In [ ]:
#rename_columns
df_cent.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

Save out to CSV files

In [ ]:
df_pc.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_pc_punishment_weighted_median.csv'), header = True, index = None)

In [ ]:
df_clust.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_clustering_punishment_weighted_median.csv'), header = True, index = None)

In [ ]:
df_cent.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_centrality_punishment_weighted_median.csv'), header = True, index = None)

# Check Partition Stability in Random Split Half

In [ ]:
from random import sample 

In [ ]:
random.shuffle(file_list)
file_list1 = file_list[:42]
file_list2 = file_list[42:]

In [ ]:
#Test Partition in mean from file list1
my_dict={}
for item in file_list1:
    name=item.split('/')[7].split('.')[0]
    #print(name)
    my_dict.setdefault(name, []).append(item)
    

data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

        
cor_dict1={}
for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict1[key] = cor_matrix

GRAPHS1 = make_graphs(cor_dict1, "positive", 0)
mean_graph = make_total_graphs(GRAPHS1['correlations'])
mean_graph_mat = np.matrix(mean_graph)
meanG1 = nx.from_numpy_matrix(mean_graph_mat)
partition1 = community.best_partition(meanG1)
partition1

In [ ]:
#Test Partition in mean from file list2
my_dict={}
for item in file_list2:
    name=item.split('/')[7].split('.')[0]
    #print(name)
    my_dict.setdefault(name, []).append(item)
    

data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

        
cor_dict2={}
for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict2[key] = cor_matrix

GRAPHS2 = make_graphs(cor_dict2, "positive", 0)
mean_graph = make_total_graphs(GRAPHS2['correlations'])
mean_graph_mat = np.matrix(mean_graph)
meanG2 = nx.from_numpy_matrix(mean_graph_mat)
partition2 = community.best_partition(meanG2)
partition2

# Brain graphs  
http://visbrain.org/index.html

In [ ]:
#positions = pd.read_csv(os.path.join(basepath,'betaseries_bevel/3_pull_timeseries/betaseries_rois.txt'), sep='\t')

In [ ]:
#positions=positions.drop('Region', axis=1)


In [ ]:
#positions_dict=positions.to_dict('index')

In [ ]:
#nx.set_node_attributes(meanG, positions_dict, 'pos')

In [ ]:
def brain_graph(G, strength, radius, colormap):
    mods=[node[1]['modules'] for node in G.nodes(data=True)]
    ccol=[]
    for m in mods:
        if m == 0.0:
            ccol.append('mediumvioletred')
        if m == 1.0:
            ccol.append('orange')
        if m == 2.0:
            ccol.append('yellowgreen')
        if m == 3.0:
            ccol.append('slateblue')
            
    # Colorbar default arguments. See `visbrain.objects.ColorbarObj`
    CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3., ndigits=1,
                  rect=(-.3, -2., 1., 4.))
        
    nodes, edges = np.asarray([list(node[1]['pos'].values()) for node in meanG.nodes(data=True)]), nx.to_numpy_array(G)

    #Change backgrounnd color here
    sc = SceneObj(size=(500, 600), bgcolor = 'grey')
    
    # Coloring method
    color_by = 'strength'
    # Because we don't want to plot every connections, we only keep connections
    select = edges > strength
    
    # Define the connectivity object
    c_default = ConnectObj('default', nodes, edges, select=select, line_width=3., cmap=colormap, color_by=color_by)
    
    # Then, we define the sources
    s_obj = SourceObj('sources', nodes, color=ccol, radius_min=radius)
    sc.add_to_subplot(c_default, title='Node Connectivity Grouped by Module')
    
    # And add connect, source and brain objects to the scene
    sc.add_to_subplot(s_obj)
    sc.add_to_subplot(BrainObj('B3'), use_this_cam=True)
    
    #Add colorbar
    cb_parr = ColorbarObj(c_default, cblabel='Connectivity strength', **CBAR_STATE, cmap=colormap)
    sc.add_to_subplot(cb_parr, row=0, col=1, width_max=200)
    
    # Coloring method
    color_by = 'count'
    
    # Define the connectivity and source object
    c_count = ConnectObj('default', nodes, edges, select=select, line_width=4.,
                         color_by=color_by, antialias=True,  cmap=colormap)

    s_obj_c = SourceObj('sources', nodes, color=ccol, radius_min=radius,
                        symbol='square')

    # And add connect, source and brain objects to the scene
    sc.add_to_subplot(c_count, row=1, col=0,
                      title='Node Grouped by Number of Connections per node')

    sc.add_to_subplot(s_obj_c, use_this_cam=True, row=1, col=0)
    sc.add_to_subplot(BrainObj('B3'), use_this_cam=True, row=1, col=0)
    
    
    #Add colorbar
    cb_parr_c = ColorbarObj(c_count, cblabel='Number of connections per node', **CBAR_STATE, cmap=colormap)
    sc.add_to_subplot(cb_parr_c, row=1, col=1, width_max=200)

    sc.preview()

    #sc.screenshot(os.path.join(basepath,'betaseries_bevel/5_analysis/example_brain_obj.png'), transparent=True)

In [ ]:
#brain_graph(meanG, .4, 15., 'Blues')

## Save GRAPHS dictionary in a pickle file in case of crash

In [ ]:
pickle.dump(GRAPHS, open(os.path.join(basepath, 'betaseries_bevel/tmp/Graphs'), 'wb'), protocol=4)

In [ ]:
with open(os.path.join(basepath, 'betaseries_bevel/tmp/Graphs'), 'rb') as pickle_file:
    try:
        while True:
            GRAPHS = pickle.load(pickle_file)
#             print (GRAPHS)
    except EOFError:
        pass